# Set up

In [88]:
!git clone https://github.com/NLP-Reichman/assignment_1.git
!mv assignment_1/data data
!rm assignment_1/ -r

Cloning into 'assignment_1'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 150 (delta 42), reused 33 (delta 25), pack-reused 92
Receiving objects: 100% (150/150), 6.79 MiB | 17.77 MiB/s, done.
Resolving deltas: 100% (63/63), done.
mv: cannot move 'assignment_1/data' to 'data/data': Directory not empty


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [89]:
import json
import os
import math
import collections
from google.colab import files
import pandas as pd

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

Note - do NOT lowercase the sentences in whi HW.

In [90]:
def get_tweet_texts(file_path: str) -> list[str]:
  with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
    data = json.loads(content)
    tweet_texts = data['tweet_text'].values()

  return tweet_texts


def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  vocabulary = set()

  for filename in os.listdir('/content/data'):
        file_path = os.path.join('/content/data', filename)
        if file_path.endswith('.json'):
          tweet_texts = get_tweet_texts(file_path)
          for tweet in tweet_texts:
              vocabulary.update(tweet)

  vocabulary.update(['א', 'ת'])

  return list(vocabulary)

print(preprocess())

['슈', '┻', '과', '🍵', '↛', '😆', '⋅', '▶', '🔘', '📊', '影', '♩', '헌', '맨', '😨', '📻', '♡', '터', '́', '🤙', '^', '挨', '午', '💃', '†', '✊', '◆', '˘', 'ｐ', '©', '🙏', '제', '\u2066', '½', '🎆', '헨', '🐮', '❥', '］', '📌', '룰', '🌅', 'ˡ', '世', 'М', '￼', '엣', '༻', 'う', '🇭', '🔛', '🈵', 'ถ', '연', '🌒', '“', '❞', '😜', '那', '💢', '🎯', '中', '🐭', '🚨', '❁', '✳', '🏔', 'ｚ', 'น', '💜', '윤', '본', 'Ò', '뱀', '유', '🐦', '🚗', '🌽', '갓', '🌎', '╴', '🏙', '현', 'н', '🐜', '◑', 'ي', 'H', '♻', 'ุ', '🤑', '🐢', '🥅', '😂', '🔵', '🍴', '✴', 'ᵗ', '🐸', '♍', '☓', 'v', '🇾', '🎥', 'ω', 'ท', 'A', '²', '✶', '🔹', '🔁', 'º', 'ゴ', '🍽', 'q', '💀', '🛴', '🤞', '🐣', 'ю', '🙌', '先', '버', '⛄', '💑', '싸', '🐟', 'u', 'υ', '🌍', '🍦', 'ë', '文', '🏆', '🈷', '🅰', '/', 'ね', '}', 'く', '🙉', '画', 'ㅏ', 'Ｌ', '👆', 'п', '😘', '▔', '두', 'ร', '드', '•', '༎', '➌', '사', '파', 'Œ', '☆', '🐝', '🌥', 'г', '⸅', 'ᵛ', 'ย', '姿', '잘', '🍯', 'せ', 'ة', '⁷', '🚼', '⛳', '🏴', '😫', 'い', '👺', '⏰', '📬', 'ศ', 'W', '▼', '🇧', 'Р', '🍪', 'ᵖ', '☑', '온', '😢', '👖', '🥂', 'و', '👂', '🖼', '😦', '👓', '⚖', '넷', 'ノ', '🏟',

## Part 2
Implement the function *lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [140]:
def create_unigram_model(tweet_texts, smoothed, vocabulary):
    model = {'א': len(tweet_texts), 'ת': len(tweet_texts)}
    total_characters = sum((len(tweet)+2) for tweet in tweet_texts)

    for tweet in tweet_texts:
      for char in tweet:
        if char in model:
            model[char] += 1
        else:
            model[char] = 1

    for char in model:
        model[char] = model[char] / total_characters

    if smoothed:
        model['<unk>'] = 1 / len(vocabulary)
    return model

def add_n_gram(model, counts, n_gram, suffix):
  if n_gram in model.keys():
    if suffix in model[n_gram]:
      model[n_gram][suffix] += 1
    else:
      model[n_gram][suffix] = 1
    counts[n_gram] += 1
  else:
    model[n_gram] = {suffix: 1}
    counts[n_gram] = 1

def lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
  '''
  Return a language model for the given lang and n_gram (n)
  :param lang: the language of the model
  :param n: the n_gram value
  :return: a dictionary where the keys are n_grams and the values are dictionaries
  '''
  model = {}
  counts = {}
  vocabulary = preprocess()

  file_path = os.path.join('/content/data', f'{lang}.json')
  tweet_texts = get_tweet_texts(file_path)

  if n == 1:
    return create_unigram_model(tweet_texts, smoothed, vocabulary)

  for tweet in tweet_texts:
    tweet = "א" * (n-1) + tweet + "ת"
    for i in range(len(tweet)-n+1):
        n_gram = tweet[i:i+n-1]
        suffix = tweet[i+n-1]
        add_n_gram(model, counts, n_gram, suffix)

  for n_gram, followers in model.items():
    for suffix in followers:
      if smoothed:
        model[n_gram][suffix] = (model[n_gram][suffix] + 1) / (counts[n_gram] + len(vocabulary))
      else:
        model[n_gram][suffix] = model[n_gram][suffix] / counts[n_gram]

  if smoothed:
    model['<unk>'] = 1 / len(vocabulary)

  return model


print(lm('en', 1, True))

{'א': 0.010404015869160392, 'ת': 0.010404015869160392, 'R': 0.009852296517343881, 'T': 0.012585447100870954, ' ': 0.13628277640908198, '@': 0.009886995847646808, 'O': 0.00480007402523798, 'N': 0.003984639763119239, 'H': 0.0037683472708976715, 'E': 0.004626577373723354, 'P': 0.0035520547786761047, 'l': 0.027951467203349643, 'D': 0.003939530633725436, ':': 0.012956729935112252, 'B': 0.003858565529685277, 'o': 0.05263078755913344, 'y': 0.015909642943891182, 'f': 0.011044796835421077, 'r': 0.03567438148443735, 'i': 0.04097181257735059, 'e': 0.06718137339949339, 'n': 0.03939761962594122, 'd': 0.019728882565899813, 's': 0.04193876724845877, 't': 0.0636038724452618, 'h': 0.03140173727980383, 'a': 0.04990342019732352, 'k': 0.008338249071792914, 'p': 0.01844385070034815, 'c': 0.022158992331448003, 'u': 0.01918988630186104, 'g': 0.013568594792787166, 'w': 0.011410296447945221, 'm': 0.016614039349040564, '😁': 2.3132886868616768e-05, '☺': 1.619302080803174e-05, '️': 0.0003666562568675758, 'b': 0.0

## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

In [142]:

def calc_prob(model, n_gram, suffix, vocabulary_size, n):
  if n == 1:
    if n_gram not in model:
      if '<unk>' in model: # smoothed
        prob = model['<unk>']
      else:
        return 0
    else:
      prob = model[n_gram]

  elif n_gram in model.keys() and suffix in model[n_gram]:
    prob = model[n_gram][suffix]
  elif '<unk>' in model: # smoothed
    prob = model['<unk>']
  else:
    return 0 # not smoothed

  return math.log(prob, 2)

def eval(model: dict, target_lang: str, n: int) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :return: the perplexity value
  '''
  perplexities = []
  vocabulary_lang = set()
  file_path = os.path.join('/content/data', f'{target_lang}.json')
  tweet_texts = get_tweet_texts(file_path)
  for tweet in tweet_texts:
    vocabulary_lang.update(tweet)
  vocabulary_lang = list(vocabulary_lang)
  vocabulary_size = len(vocabulary_lang)

  for tweet in tweet_texts:
    total_log_prob = 0
    tweet = "א" * (n-1) + tweet + "ת"
    N = len(tweet) - n

    for i in range(N-1):
      if n == 1:
        n_gram = tweet[i]
      else:
        n_gram = tweet[i:i+n-1]
      suffix = tweet[i+n-1]
      total_log_prob += calc_prob(model, n_gram, suffix, vocabulary_size, n)

    perplexities.append(2 ** (-total_log_prob / N))

  return sum(perplexities) / len(perplexities)

# round(eval(lm('en', 1, True), 'en', 1))

39

## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [143]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''
  results = []
  files_list = list(filter(lambda f: f.endswith('.json'), os.listdir('/content/data')))
  lang_list = list(map(lambda f: f[:-5], files_list))

  for source in lang_list:
    for n in range(1,5):
      model = lm(source, n, True)


      for target in lang_list:
          perplexity = eval(model, target, n)
          results.append({
              'source': source,
              'target': target,
              'n': n,
              'perplexity': perplexity
          })

  return pd.DataFrame(results)


match()

KeyboardInterrupt: 

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution.

In [105]:
import random

def sample_from_dist(tokens, probs):
    # Calculate the cumulative probability
    cum_prob = 0
    cum_probabilities = []
    for p in probs:
        cum_prob += p
        cum_probabilities.append(cum_prob)

    # Draw a random number in [0, max(cum_probabilities)]
    r = random.uniform(0, max(cum_probabilities))

    # Find where the random number fits in the cumulative distribution
    for i, cp in enumerate(cum_probabilities):
        if r < cp:
            chosen_char = tokens[i]
            break

    return chosen_char

def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  text = prompt
  model = lm(lang, n, True)
  n_gram = prompt[-n+1:]

  for _ in range(number_of_tokens):
    if n == 1 :
      distribution = model
    else:
      distribution = model[n_gram]
    tokens = list(distribution.keys())
    probs = list(distribution.values())
    gen_char = sample_from_dist(tokens, probs)
    text += gen_char

    if text.endswith('ת'):
      break
    n_gram = n_gram[1:]
    n_gram += gen_char

  return text


generate('en', 4, "I eat", 20, 5)

'I eatnight bilizayn: thro'

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [144]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
####################
# PLACE TESTS HERE #

####################
# Create tests


# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_lm():
    return {
        'english_2_gram_length': len(lm('en', 2, True)),
        'english_3_gram_length': len(lm('en', 3, True)),
        'french_3_gram_length': len(lm('fr', 3, True)),
        'spanish_3_gram_length': len(lm('es', 3, True)),
    }

def test_eval():
    return {
        'en_en': eval(lm('en', 3, True), 'en', 3),
        'en_fr': eval(lm('en', 3, True), 'fr', 3),
        'en_tl': eval(lm('en', 3, True), 'tl', 3),
        'en_nl': eval(lm('en', 3, True), 'nl', 3),
    }

def test_match():
    df = match()
    return {
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_1_gram': generate('en', 1, "I", 20, 5),
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
files.download('results.json')
!ls -l

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

total 12
drwxr-xr-x 3 root root 4096 Apr 16 12:06 data
-rw-r--r-- 1 root root  942 Apr 16 13:23 results.json
drwxr-xr-x 1 root root 4096 Apr 12 13:22 sample_data
